### Apı key tanımlıyoruz.

In [2]:
import os 
from secret_key import openapi_key
os.environ["OPEN_API_KEY"] = openapi_key

### LLM import ediyoruz.

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(temperature = 0.6)
name = llm("I want to open a restaurant for Indian food.Suggest a fency name for this")
print(name)

### PromptTemplate kullanıyoruz.

In [7]:
from langchain_core.prompts import PromptTemplate

template = "I want to open a restaurant for {cuisine} food.Suggest a fency name for this"

PromptTemplate_name = PromptTemplate(
    input_variables=["cuisine"],
    template=template
)
PromptTemplate_name.format(cuisine = "Italian")

'I want to open a restaurant for Italian food.Suggest a fency name for this'

### Chain ile prompt_template ve llm birlestiriyoruz.

In [8]:
from langchain.chains import LLMChain

# chain = LLMChain(llm = llm,prompt = PromptTemplate_name)
chains = PromptTemplate_name | llm
chains.invoke("American")

'\n\n"Stateside Bistro"'

In [ ]:
llm = OpenAI(temperature= 0.6)
PromptTemplate_name = PromptTemplate(
    input_variables=["cuisine"],
    template="I want to open a restaurant for {cuisine} food.Suggest a fency name for this"
)
chain = LLMChain(llm = llm,prompt = PromptTemplate_name)

PromptTemplate_items = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggest some menu items for {restaurant_name}.Return it as a comma separated list"
)
food_items_chain = LLMChain(llm = llm,prompt = PromptTemplate_items)


In [16]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [chain,food_items_chain])
response = chain.invoke("Indian")
print(response)

{'input': 'Indian', 'output': '\n\n1. Spicy Chicken Curry\n2. Vegetable Samosas\n3. Tandoori Shrimp\n4. Lamb Vindaloo\n5. Paneer Tikka Masala\n6. Aloo Gobi (potato and cauliflower curry)\n7. Garlic Naan\n8. Mango Lassi\n9. Chicken Biryani\n10. Chana Masala (chickpea curry)\n11. Palak Paneer (spinach and cottage cheese curry)\n12. Tandoori Chicken\n13. Onion Bhaji (fried onion fritters)\n14. Dal Makhani (creamy lentil curry)\n15. Vegetable Korma\n16. Raita (yogurt dip)\n17. Gulab Jamun (sweet fried dough balls in syrup)\n18. Chicken Tikka Masala\n19. Malai Kofta (vegetable and cheese balls in creamy sauce)\n20. Mango Kulfi (mango ice cream)'}


In [17]:
llm = OpenAI(temperature= 0.6)
PromptTemplate_name = PromptTemplate(
    input_variables=["cuisine"],
    template="I want to open a restaurant for {cuisine} food.Suggest a fency name for this"
)
chain = LLMChain(llm = llm,prompt = PromptTemplate_name,output_key = "restaurant_name")

PromptTemplate_items = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggest some menu items for {restaurant_name}."
)
food_items_chain = LLMChain(llm = llm,prompt = PromptTemplate_items,output_key = "menu_items")

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(chains = [chain,food_items_chain],
                        input_variables = ["cuisine"],
                        output_variables = ["restaurant_name","menu_items"])
chain({"cuisine":"Arabic"})

In [2]:
from langchain.agents import initialize_agent,AgentType,load_tools
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.6)

tools = load_tools(["wikipedia","llm-math"],llm = llm)

agent = initialize_agent(
    llm = llm,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)
agent.invoke("When was Elon musk born? What is his age right now in 2023")



> Entering new AgentExecutor chain...
 I need to find out when Elon Musk was born and then calculate his age in 2023
Action: wikipedia
Action Input: "Elon Musk"
Observation: Page: Elon Musk
Summary: Elon Reeve Musk  (; born June 28, 1971) is a businessman known for his key roles in the space company SpaceX and the automotive company Tesla, Inc. His other involvements include ownership of X Corp., the company that operates the social media platform X (formerly Twitter), and his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. Musk is the wealthiest individual in the world; as of December 2024, Forbes estimates his net worth to be US$344 billion.
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at the age of 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later trans

{'input': 'When was Elon musk born? What is his age right now in 2023',
 'output': '52'}

### MEMORY

In [10]:
from langchain.chains import LLMChain
chain = LLMChain(llm = llm,prompt = PromptTemplate_name)
name = chain.invoke("Türkiye")
print(name)

{'cuisine': 'Türkiye', 'text': '\n\n"Ottoman Palace Cuisine"'}


In [18]:
type(chain.memory)

NoneType

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
chain = LLMChain(llm = llm,prompt = PromptTemplate_name,memory = memory)
name = chain.invoke("Mexican")
print(name)

In [22]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Jardín de Sabores" (The Garden of Flavors) 


In [ ]:
from langchain.chains import ConversationChain
convo = ConversationChain(llm = OpenAI(temperature=0.7))
print(convo.prompt.template)

In [27]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=3)

convo = ConversationChain(
    llm = OpenAI(temperature=0.6),
    memory = memory
)
convo.invoke("Who won the first cricket world cup?")

{'input': 'Who won the first cricket world cup?',
 'history': '',
 'response': " The first cricket world cup was held in 1975 and was won by the West Indies team. They beat Australia in the finals by 17 runs. The match was held at Lord's Cricket Ground in London, England. The West Indies team was led by Clive Lloyd and their star players included Viv Richards, Gordon Greenidge, and Andy Roberts. The tournament consisted of eight teams, including India, Pakistan, and England. The West Indies team dominated the tournament, winning all of their matches and becoming the first ever cricket world cup champions. "}

In [25]:
convo.invoke("what is 5+5")

{'input': 'what is 5+5',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was held in 1975 and was won by the West Indies team. The final match was played between the West Indies and Australia, with the West Indies winning by 17 runs. The man of the match was West Indies' Clive Lloyd, who scored 102 runs.",
 'response': ' 5+5 is 10.'}

In [28]:
convo.invoke("who was the captain of the winning team?")

{'input': 'who was the captain of the winning team?',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was held in 1975 and was won by the West Indies team. They beat Australia in the finals by 17 runs. The match was held at Lord's Cricket Ground in London, England. The West Indies team was led by Clive Lloyd and their star players included Viv Richards, Gordon Greenidge, and Andy Roberts. The tournament consisted of eight teams, including India, Pakistan, and England. The West Indies team dominated the tournament, winning all of their matches and becoming the first ever cricket world cup champions. ",
 'response': ' The captain of the winning team was Clive Lloyd. He was a left-handed batsman and a medium-pace bowler. He led the West Indies team to two consecutive world cup victories in 1975 and 1979. He was known for his aggressive and fearless leadership style on the field. He also had an impressive batting record, scoring over 7,500 runs in